# SkillFactory
## Введение в ML, введение в sklearn

В этом задании мы с вами рассмотрим данные с конкурса [Задача предсказания отклика клиентов ОТП Банка](http://www.machinelearning.ru/wiki/index.php?title=%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BF%D1%80%D0%B5%D0%B4%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D0%B8%D1%8F_%D0%BE%D1%82%D0%BA%D0%BB%D0%B8%D0%BA%D0%B0_%D0%BA%D0%BB%D0%B8%D0%B5%D0%BD%D1%82%D0%BE%D0%B2_%D0%9E%D0%A2%D0%9F_%D0%91%D0%B0%D0%BD%D0%BA%D0%B0_%28%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%29)

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

In [29]:
df_train = pd.read_csv('data/otp_train.csv', sep='\t', encoding='utf8')

In [30]:
df_train.shape

(15223, 52)

In [31]:
df_test = pd.read_csv('data/otp_test.csv', sep='\t', encoding='utf8')

In [32]:
df_test.shape

(14910, 52)

In [34]:
df_train.loc[:, 'sample'] = 'train'
df_test.loc[:, 'sample'] = 'test'

In [35]:
df = df_test.append(df_train).reset_index(drop=True)

In [36]:
def preproc_data(df_input):
    df_output = df_input.copy()
    
    # Перечень колонок, которые нужно удалить.
    # Колонки с адресами вероятнее всего дублируются, 
    # поэтому оставим только FACT_ADDRESS_PROVINCE, поскольку фактический адрес
    # более важен, чем адрес регистрации
    # Удаляем столбец с регионами, так как субъекты входят в регионы
    remove_cols = ['AGREEMENT_RK',
                   'REG_ADDRESS_PROVINCE',
                   'POSTAL_ADDRESS_PROVINCE',
                   'TP_PROVINCE',
                   'REGION_NM']
    df_output = df_output.drop(remove_cols, axis=1)
    
    
    # Перечень столбцов, значения которых нужно конвертировать из str во float
    numbers_cols = ['PERSONAL_INCOME',
                    'CREDIT',
                    'FST_PAYMENT',
                    'LOAN_AVG_DLQ_AMT',
                    'LOAN_MAX_DLQ_AMT']
    for nc in numbers_cols:
        df_output[nc] = df_output[nc].map(lambda x: x.replace(',', '.')).astype('float')
        
    # Дополнительный one-hot-vector для учета людей, не имеющих работы
    df_output['НЕ РАБОТАЕТ'] = df_output['GEN_TITLE'].isna().replace(False, 0).replace(True, 1)
    
    # Переводим категории FAMILY_INCOME в числа от 0 до 4 с помощью метода replace
    di = {'до 5000 руб.': 0, 
          'от 5000 до 10000 руб.': 1,
          'от 10000 до 20000 руб.': 2,
          'от 20000 до 50000 руб.': 3,
          'свыше 50000 руб.': 4
           }
    df_output['FAMILY_INCOME'] = df_output['FAMILY_INCOME'].replace(di)
    
    # Заменяем пропуски на нули в колонке PREVIOUS_CARD_NUM_UTILIZED
    df_output['PREVIOUS_CARD_NUM_UTILIZED'] = df_output['PREVIOUS_CARD_NUM_UTILIZED'].fillna(0)
    
    # Заменяем пропуски в поле WORK_TIME на Медиану
    MEDIAN = df_output['WORK_TIME'].dropna().median()
    df_output['WORK_TIME'] = df_output['WORK_TIME'].map(lambda x: MEDIAN if np.isnan(x) else x)
    
    # Перечень категориальных признаков, 
    # и разделяем one-hot-vectors с помощью метода get_dummies. По умолчанию get_dummies
    # проставит значения 0 по тем строкам, в которых пропущены значения
    categorize_cols = ['EDUCATION',
                       'MARITAL_STATUS',
                       'GEN_INDUSTRY',
                       'GEN_TITLE',
                       'ORG_TP_STATE',
                       'ORG_TP_FCAPITAL',
                       'JOB_DIR',
                       'FACT_ADDRESS_PROVINCE']
    # конвертируем временно 'sample' в числа, иначе метод выдает ошибку
    df_output['sample'] = df_output['sample'].map(lambda x: 1 if x == 'train' else 0)
    df_output = pd.get_dummies(df_output)
    df_output['sample'] = df_output['sample'].map(lambda x: 'train' if x == 1 else 'test')
    
    return df_output

In [37]:
df_preproc = df.pipe(preproc_data)

df_train_preproc = df_preproc.query('sample == "train"').drop(['sample'], axis=1)
df_test_preproc = df_preproc.query('sample == "test"').drop(['sample'], axis=1)

#### Задание 4. Отделите целевую переменную и остальные признаки

Должно получится:
* 2 матрицы: X и X_test
* 2 вектора: y и y_test

In [38]:
X      = df_train_preproc.drop(['TARGET'], axis=1)
X_test = df_test_preproc.drop(['TARGET'], axis=1)

y      = df_train_preproc['TARGET']
y_test = df_test_preproc['TARGET']

#### Задание 5. Обучение и оценка качества разных моделей

In [39]:
from sklearn.model_selection import train_test_split
# test_size=0.3, random_state=42

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

## Your Code Here
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

# Приведем параметры к единому масштабу


In [40]:
print(X_train.shape[1])

194


In [41]:
def try_params(hidden_size, learning_rate, num_epoch, weight_decay):
    
    X      = df_train_preproc.drop(['TARGET'], axis=1)
    X_test = df_test_preproc.drop(['TARGET'], axis=1)

    y      = df_train_preproc['TARGET']
    y_test = df_test_preproc['TARGET']
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)
    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    
    
    X_valid = ss.transform(X_valid)

    import torch
    import torch.nn as nn
    import torch.nn.functional as F

    #hyperparameters
    hidden_size = 100
    learning_rate = 0.01
    num_epoch = 500


    class IrisNN(nn.Module):
        def __init__(self):
            super(IrisNN, self).__init__()
            self.fc1 = nn.Linear(X_train.shape[1], hidden_size)
            self.fc2 = nn.Linear(hidden_size, hidden_size)
            self.fc3 = nn.Linear(hidden_size, 2)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    net = IrisNN()

    #choose optimizer and loss function
    criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, betas=(0.99, 0.999), eps=1e-08, weight_decay=weight_decay, amsgrad=False)

    loss_value = 0

    accuracy = 0

    for epoch in range(num_epoch):
        X = torch.FloatTensor(X_train)

        y = torch.LongTensor(y_train)

        optimizer.zero_grad()

        out = net(X)

        loss = criterion(out, y)

        loss.backward()

        optimizer.step()

        xtest = torch.FloatTensor(X_valid)

        ytest = torch.LongTensor(y_valid)

        test_out = net(xtest)
        _, predicted = torch.max(test_out.data, 1)

        correct = (predicted == ytest).sum()

        accuracy = 100 * float(correct) / len(ytest)

        loss_value = loss.item()

    params = ('Epochs: {}, LR: {}, Hidden_size: {}, weight_decay: {} Loss: {}, test {}'.format(num_epoch, learning_rate, hidden_size, weight_decay, loss_value, accuracy))
    return accuracy, params

In [ ]:
accuracy_max = 0

for i in range(10):
    accuracy, params = try_params(hidden_size=100, learning_rate=0.001, num_epoch=100, weight_decay=1e-4)
    if accuracy > accuracy_max:
        accuracy_max = accuracy
        print(params)